In [ ]:
!pip install vllm ray

In [1]:
0

0

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

import torch

torch.cuda.device_count()
os.environ["CUDA_VISIBLE_DEVICES"]

'0,1'

In [3]:
import ray
from vllm import LLM

# ray.init(num_gpus=2) # initialize ray before LLM so that GPUs are defined

# llm_mistral = LLM(model="mistralai/Mistral-7B-Instruct-v0.1", tensor_parallel_size=1) 
llm_asclepius_13b = LLM(model="starmpcc/Asclepius-13B", tensor_parallel_size=1)
# llm_asclepius_7b = LLM(model="starmpcc/Asclepius-7B", tensor_parallel_size=1)

# mistralai/Mistral-7B-Instruct-v0.1 | GPU blocks: 9257, # CPU blocks: 2048
# starmpcc/Asclepius-13B             | GPU blocks: 874,  # CPU blocks: 327
# starmpcc/Asclepius-7B              | GPU blocks: 2869, # CPU blocks: 512

llm = llm_asclepius_13b

2023-11-01 16:34:05,214	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-01 16:34:05,657	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


INFO 11-01 16:34:05 llm_engine.py:72] Initializing an LLM engine with config: model='starmpcc/Asclepius-13B', tokenizer='starmpcc/Asclepius-13B', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=2048, download_dir=None, load_format=auto, tensor_parallel_size=1, quantization=None, seed=0)


You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


INFO 11-01 16:34:27 llm_engine.py:207] # GPU blocks: 874, # CPU blocks: 327


In [4]:
llm

In [6]:
import time

from vllm import SamplingParams

sampling_params = SamplingParams(
    n=1,
    temperature=0.1,
    top_p=0.75,
    max_tokens=512,
)

prompt_base = """You are an intelligent clinical languge model.
Below is a snippet of patient's discharge summary and a following instruction from healthcare professional.
Write a response that appropriately completes the instruction.
The response should provide the accurate answer to the instruction, while being concise.

[Discharge Summary Begin]
{note}
[Discharge Summary End]

[Instruction Begin]
{question}
[Instruction End] 
"""

notes = [
    "Indiction: History : ___F with shortness of breath, cough and recent pneumonia Findings: Heart size remains moderately enlarged. The aorta remains tortuous. Mediastinal and hilar contours are similar with prominence of the right hilum again noted. Pulmonary vasculature is not engorged. Minimal patchy lower lobe opacities, more pronounced on the left, likely reflect areas of atelectasis. No focal consolidation, pleural effusion or pneumothorax is seen. There are no acute osseous abnormalities.",
    "The patient today completed SBRT for his T-Spine. This patient received a total dose of 30Gy delivered in 5 equal treatments of 6Gy each. Treatment was delivered on 3/29/21, 4/5/21, 4/8/21, 4/14/21 and 4/19/21. We delivered the dose as measured at the 71.4% isodose.",
]
questions = [
    "What would be the impression section of the radiology report, given this findings section?",
    "What is the total dose that patient received?",
]

times = []
lengths = []
for note, question in zip(notes, questions):
    prompt = prompt_base.format(note=note, question=question)
    s0 = time.perf_counter()
    output = llm.generate(prompts=prompt, sampling_params=sampling_params, use_tqdm=False)
    s1 = time.perf_counter()
    lengths.append((len(prompt), len(output[0].outputs[0].text)))
    times.append(s1 - s0)

print(sum(times) / len(times))
print(sum([output_len / t for (_, output_len), t in zip(lengths, times)]) / len(times))

2.7314981315284967
145.30938675282002


In [ ]:
[(output_len) / t for (prompt_len, output_len), t in zip(lengths, times)]